# Links

In [27]:
import pandas as pd
import os
from utils.youtube_extractor import extrair_links_playlist  # Ajuste conforme o caminho do seu módulo

# URL da playlist do YouTube
url_playlist = "https://www.youtube.com/playlist?list=PLcwPvhFHJ_F83aRklVj8vc6f4zIZafMKn"

# Obter nome da playlist e informações dos vídeos
nome_playlist, videos_info = extrair_links_playlist(url_playlist)

# Converter a lista de dicionários em DataFrame
videos_df = pd.DataFrame(videos_info)

# Exibindo os resultados
print("Nome da Playlist:", nome_playlist)
videos_df.head()

100%|██████████| 20/20 [00:26<00:00,  1.31s/it]

Nome da Playlist: Itaú Views | 5ª Temporada


,titulo,url,duração,visualizações,avaliação,autor,descrição,data_publicação,curtidas,keywords,número_curtidas,número_descurtidas
0,Privatização da Sabesp: o que esperar pela fre...,https://youtube.com/watch?v=nlpEgrjDC-8,1941,356,None,Itaú Views | 5ª Temporada,None,2023-12-18,None,[],None,None
1,Qual o melhor trade com renda fixa?,https://youtube.com/watch?v=8nxRcMYud8g,1769,621,None,Itaú Views | 5ª Temporada,None,2023-12-11,None,[],None,None
2,Um raio-X da China,https://youtube.com/watch?v=Wp5mBfznTyk,1299,402,None,Itaú Views | 5ª Temporada,None,2023-12-04,None,[],None,None
3,AI e a corrida da tecnologia,https://youtube.com/watch?v=Kr9z9GofAZw,2040,246,None,Itaú Views | 5ª Temporada,None,2023-11-27,None,[],None,None
4,Ibovespa em 145 mil pontos,https://youtube.com/watch?v=bq5Q_Fjd8Ms,1653,617,None,Itaú Views | 5ª Temporada,None,2023-11-20,None,[],None,None


In [33]:
# Salvar links
videos_df.to_csv(os.path.join("../data/links/",f"links_{nome_playlist}.csv"), index=False)

# Video

In [2]:
from langchain.document_loaders.blob_loaders.youtube_audio import YoutubeAudioLoader
from langchain_community.document_loaders.generic import GenericLoader
from langchain_community.document_loaders.parsers import (
    OpenAIWhisperParser,
)
from langchain.document_loaders.parsers.audio import OpenAIWhisperParserLocal

In [3]:
# set a flag to switch between local and remote parsing
# change this to True if you want to use local parsing
local = True

In [4]:
import pandas as pd
videos_df = pd.read_csv("../data/links/links_Itaú Views | 5ª Temporada.csv")
url_menor_video = videos_df[videos_df['duração'] == videos_df['duração'].min()].url.values[0]

In [5]:
from transformers import WhisperProcessor
processor = WhisperProcessor.from_pretrained("openai/whisper-small")
processor.get_decoder_prompt_ids(language="portuguese", task="transcribe")

/home/vicosbe/.local/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


[(1, 50267), (2, 50359), (3, 50363)]

In [6]:
# Two Karpathy lecture videos
urls = videos_df.url.values.tolist() # ['https://youtube.com/watch?v=plW9ZiMFcOM']  #[url_menor_video]

# Directory to save audio files
save_dir = "../data/videos/"


# Transcribe the videos to text
if local:
    loader = GenericLoader(
        # exemplo "../data/videos/Balanço dos Balanços 2T23.m4a!" necessario passar a lingua por conta do erro: Multiple languages detected when trying to predict the most likely target language for transcription. It is currently not supported to transcribe to different languages in a single batch. Please make sure to either force a single language by passing `language='...'` or make sure all input audio is of the same language.
        YoutubeAudioLoader(urls, save_dir), 
        
        OpenAIWhisperParserLocal(chunk_length=20, # quando chunk=30, houve alguns bugs, palavras repetidas na transcrição
                                                                     forced_decoder_ids=processor.get_decoder_prompt_ids(language="portuguese", task="transcribe")
                                                                     )
        
    )
else:
    loader = GenericLoader(YoutubeAudioLoader(urls, save_dir), OpenAIWhisperParser())
docs = loader.load()

Using the following model:  openai/whisper-small


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


[youtube] Extracting URL: https://youtube.com/watch?v=plW9ZiMFcOM
[youtube] plW9ZiMFcOM: Downloading webpage
[youtube] plW9ZiMFcOM: Downloading ios player API JSON
[youtube] plW9ZiMFcOM: Downloading android player API JSON


[youtube] plW9ZiMFcOM: Downloading m3u8 information
[info] plW9ZiMFcOM: Downloading 1 format(s): 140
[download] ../data/videos//Balanço dos Balanços 2T23.m4a has already been downloaded
[download] 100% of   28.54MiB
[ExtractAudio] Not converting audio ../data/videos//Balanço dos Balanços 2T23.m4a; file is already in target format m4a
Transcribing part ../data/videos/AI e a corrida da tecnologia.m4a!
Transcribing part ../data/videos/Balanço dos Balanços 2T23.m4a!
Transcribing part ../data/videos/Buy Side & Sell Side ｜ Agro, Aço, Mineração, Papel & Celulose.m4a!
Transcribing part ../data/videos/Buy Side & Sell Side ｜ Fundos Imobiliários.m4a!
Transcribing part ../data/videos/Buy Side & Sell Side ｜ Óleo & Gás.m4a!
Transcribing part ../data/videos/Como investir em créditos privados.m4a!
Transcribing part ../data/videos/El Niño： Impactos e Oportunidades para os investidores.m4a!
Transcribing part ../data/videos/Especial Macro Vision： um olhar sobre a bolsa brasileira.m4a!
Transcribing 

/home/vicosbe/.local/lib/python3.11/site-packages/transformers/pipelines/base.py:1123: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


...

/home/vicosbe/.local/lib/python3.11/site-packages/transformers/pipelines/base.py:1123: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


Transcribing part ../data/videos/Tensões globais e os efeitos para o Brasil.m4a!


/home/vicosbe/.local/lib/python3.11/site-packages/transformers/pipelines/base.py:1123: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


Transcribing part ../data/videos/Um raio-X da China.m4a!


/home/vicosbe/.local/lib/python3.11/site-packages/transformers/pipelines/base.py:1123: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


In [12]:
docs[0]

Document(page_content=' Oi pessoal, aqui é Marcelo Serrâneo, esse é mais um episódio do podcast Talvius. Antes de começar, eu deixo convite pra você nos acompanhar também nas outras plataformas. A gente tem criado alguns conteúdos complementares no Instagram e a gente disponibiliza todos os relatórios aqui do research da nossa área de pesquisa macro no Telegram. Deixando mais um convite, as edições do podcast agora tão em vídeo tanto no youtube quanto no próprio Spotify onde a gente estreia hoje a edição desse podcast em vídeo bom o assunto de hoje é tecnologia a ideia a gente falar um pouco sobre as tendências de investimentos em tecnologia, falar sobre AI, que é o tema que domina um pouco esse assunto, a gente vai entender um pouco como os convidados aqui presentes têm olhado para a BigTex, quais são as top picks quando a gente olha para esses segmentos e entender se há espaço para mais valorização ainda desse setor comparativamente ao índice e ao mercado como um tudo. Então para col

In [5]:
from utils.document_manager import save_docs_to_csv, load_docs_from_csv
# Exemplo de uso das funções
csv_file_path = '../data/transcripts/transcripts.csv'

In [14]:
# Supondo que 'docs' é a lista de documentos que você deseja salvar
save_docs_to_csv(docs, csv_file_path)


In [8]:
# Carregar os documentos de volta para o formato LangChain Document
loaded_docs = load_docs_from_csv(csv_file_path)
loaded_docs[0]

Documentos carregados de ../data/transcripts/transcripts.csv


Document(page_content=' Oi pessoal, aqui é Marcelo Serrâneo, esse é mais um episódio do podcast Talvius. Antes de começar, eu deixo convite pra você nos acompanhar também nas outras plataformas. A gente tem criado alguns conteúdos complementares no Instagram e a gente disponibiliza todos os relatórios aqui do research da nossa área de pesquisa macro no Telegram. Deixando mais um convite, as edições do podcast agora tão em vídeo tanto no youtube quanto no próprio Spotify onde a gente estreia hoje a edição desse podcast em vídeo bom o assunto de hoje é tecnologia a ideia a gente falar um pouco sobre as tendências de investimentos em tecnologia, falar sobre AI, que é o tema que domina um pouco esse assunto, a gente vai entender um pouco como os convidados aqui presentes têm olhado para a BigTex, quais são as top picks quando a gente olha para esses segmentos e entender se há espaço para mais valorização ainda desse setor comparativamente ao índice e ao mercado como um tudo. Então para col

In [ ]:
# chat https://python.langchain.com/docs/integrations/document_loaders/youtube_audio/